In [3]:
import numpy as np 

### E1: Generate a random MPS for a system of 24 qubits with bond dimension 10

In [4]:
# My ordering of legs is virtual left,physical,virtual right

def random_mps(L,chi):
    mps_list = []
    mps_list.append(np.random.rand(2,chi)+1j*np.random.rand(2,chi))

    for jj in range(1,L-1):
        mps_list.append(np.random.rand(chi,2,chi)+1j*np.random.rand(chi,2,chi))
    
    mps_list.append(np.random.rand(chi,2)+1j*np.random.rand(chi,2))


    # Some first attempt at normalization 
    for ii in range(len(mps_list)):
        mps_list[ii] = mps_list[ii]/np.linalg.norm(mps_list[ii])
        
    return mps_list


In [5]:
psi_mps = random_mps(24,10)

In [6]:
def info_mps(psi):
    print(len(psi))
    for tensor in psi:
        print(np.shape(tensor))


In [7]:
info_mps(psi_mps)

24
(2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2)


In [8]:

def fullpsi_from_mps(mps_list):

    # First tensor
    w_rec = mps_list[0]

    for jj in range(1,len(mps_list)-1):
        #print(f"{jj} - {np.shape(w_rec)} * {np.shape(mps_list[jj])}")
        w_rec = np.einsum("ia,ajb->ijb", w_rec, mps_list[jj])
        w_rec = np.reshape(w_rec, (np.shape(w_rec)[0] * np.shape(w_rec)[1], np.shape(w_rec)[2]))

    # Final tensor 
    w_rec = np.einsum("ia,aj->ij", w_rec, mps_list[-1])
    w_rec = np.reshape(w_rec, (np.shape(w_rec)[0] * np.shape(w_rec)[1]))

    return w_rec

In [9]:
psi_reconstructed = fullpsi_from_mps(psi_mps)
print(f"{np.shape(psi_reconstructed)=}")

print(f"{np.linalg.norm(psi_reconstructed)=}")
print(f"{np.sqrt(psi_reconstructed.conj().T @ psi_reconstructed)=}")

np.shape(psi_reconstructed)=(16777216,)
np.linalg.norm(psi_reconstructed)=np.float64(0.033596484583845905)
np.sqrt(psi_reconstructed.conj().T @ psi_reconstructed)=np.complex128(0.03359648458384607+0j)


### E2: Write a function that computes the overlap between two MPS in an efficient way

In [10]:
def overlap_mps(psi1, psi2):

    assert len(psi1) == len(psi2)

    overlap = np.einsum("pr,ps->rs", psi1[0], psi2[0].conj())

    for ii in range(1,len(psi1)-1):
        #print(f"{ii=}, {np.shape(overlap)=}, {np.shape(psi1[ii])=}")
        overlap = np.einsum("rs,rpt->spt", overlap, psi1[ii])
        overlap = np.einsum("spt,spq->tq", overlap, psi2[ii].conj())

    overlap = np.einsum("tq,tp->qp", overlap, psi1[-1])
    overlap = np.einsum("qp,qp->", overlap, psi2[-1].conj())

    return overlap, np.sqrt(overlap) 



In [11]:
overlap_mps(psi_mps, psi_mps)

(np.complex128(0.0011287237763926076+4.404571325722362e-20j),
 np.complex128(0.033596484583846085+6.555107446926418e-19j))

### E3: Write a function that brings an MPS to left canonical form using QR 

In [12]:
def orthogonalize_mps_left(psi_mps):
    q,r = np.linalg.qr(psi_mps[0])
    psi_mps[0] = q

    for ii in range(1,len(psi_mps)-1):
 
        r_m = np.einsum("ij,jkl->ikl", r, psi_mps[ii])
        print(f"{ii=}, {np.shape(r_m)=}")
        q, r = np.linalg.qr(np.reshape(r_m, (r_m.shape[0]*r_m.shape[1],r_m.shape[2])))
        psi_mps[ii] = np.reshape(q, (r_m.shape[0],r_m.shape[1], r.shape[0] ))

    psi_mps[-1] = np.einsum("ij,jp->ip", r, psi_mps[-1])


In [13]:
psi1 = random_mps(12, 10)
orthogonalize_mps_left(psi1)

ii=1, np.shape(r_m)=(2, 2, 10)
ii=2, np.shape(r_m)=(4, 2, 10)
ii=3, np.shape(r_m)=(8, 2, 10)
ii=4, np.shape(r_m)=(10, 2, 10)
ii=5, np.shape(r_m)=(10, 2, 10)
ii=6, np.shape(r_m)=(10, 2, 10)
ii=7, np.shape(r_m)=(10, 2, 10)
ii=8, np.shape(r_m)=(10, 2, 10)
ii=9, np.shape(r_m)=(10, 2, 10)
ii=10, np.shape(r_m)=(10, 2, 10)


In [14]:
info_mps(psi1)

12
(2, 2)
(2, 2, 4)
(4, 2, 8)
(8, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2, 10)
(10, 2)


### E4: Write a function that checks if an MPS is in left canonical form

In [15]:
def is_left_can(psi):
    is_can = []
    left_env = np.einsum("pv,pw->vw", psi1[0],psi1[0].conj())
    is_can.append(np.allclose(left_env, np.eye(left_env.shape[1])))
    for ii in range(1,len(psi)-1):
        left_env = np.einsum("lpr,lpk->rk", psi1[1],psi1[1].conj())
        is_can.append(np.allclose(left_env, np.eye(left_env.shape[1])))

    return np.all(is_can)


In [16]:
print(is_left_can(psi1))
print(overlap_mps(psi1,psi1))
print(np.einsum("vp,vp->", psi1[-1],psi1[-1].conj()))

True
(np.complex128(0.035649010558665026-5.056735236234578e-18j), np.complex128(0.18880945569188273-1.3391106969999004e-17j))
(0.03564901055866501+0j)
